In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import spacy

In [2]:
df = pd.read_pickle('../Datasets_ML/Rest_final_ML.pickle')
df.head()

,name,address,coord,keywords,embedding
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457","[44.3069541, -89.8457834]","[restaurant, takeout, dine-in, delivery, staff...","[[0.84636, -1.1249, -2.3415, -0.12289, 3.0118,..."
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...","[36.1646332, -115.0607804]","[mexican restaurant, crêperie, ice cream shop,...","[[-1.15987, -0.23539498, -2.703, -0.27396, 4.7..."
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...","[43.0576581, -87.914643]","[cajun restaurant, creole restaurant, delivery...","[[-1.72852, -3.57585, -0.14324999, -0.1046505,..."
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149","[42.8541679, -88.3142471]","[sandwich shop, caterer, fast food restaurant,...","[[-1.46857, -3.8650498, -1.97485, 2.0822, 1.41..."
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341","[41.7580941, -111.8302904]","[pizza restaurant, delivery, reservations requ...","[[0.32144, -1.7504499, -3.4772, 1.227705, 1.85..."


In [3]:
model_name = "en_core_web_lg"

def load_model(model_name): 
    try:
        nlp = spacy.load(model_name)
        print(f"Modelo {model_name} cargado exitosamente.")
        return nlp
    except OSError:
        print(f"El modelo {model_name} no está instalado. Descargando e instalando...")
        spacy.cli.download(model_name)
        nlp = spacy.load(model_name)
        return nlp
    
nlp_en = load_model(model_name)

c:\Users\guill\anaconda3\envs\ProyectML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Modelo en_core_web_lg cargado exitosamente.


In [4]:
def buscar_palabras_similares(dataframe, columna_embbeding, lista_busqueda, umbral_similitud=0.7):
    indices = []
    embeddings_busqueda = np.array([nlp_en(palabra).vector for palabra in lista_busqueda])

    for index, embedding in enumerate(dataframe[columna_embbeding]):
        if embedding is not None:
            # Calcular la similitud coseno solo si el embedding es válido (no es None)
            similitudes = cosine_similarity(embedding, embeddings_busqueda).max(axis=1)
            
            # Si la similitud es mayor que el umbral, agregar el índice
            if np.any(similitudes >= umbral_similitud):
                indices.append(index)

    return indices

In [5]:
# Lista de palabras para buscar
palabras_busqueda = ['sushi','beer']

# Buscar palabras iguales o similares en la columna 'palabras'
indices = buscar_palabras_similares(df, 'embedding', palabras_busqueda)

print("indices", list(set(indices)))

indices [0, 2048, 18435, 2053, 6, 16391, 11, 2060, 16, 2064, 18, 2067, 20, 2069, 22, 8213, 12309, 27, 16411, 2077, 34, 2084, 38, 2088, 6185, 2092, 2093, 2096, 53, 14394, 2109, 2110, 18494, 69, 18502, 2119, 72, 18503, 2124, 77, 78, 2126, 80, 2128, 2130, 2131, 8275, 12364, 86, 18511, 2136, 2139, 2142, 16478, 2144, 16479, 2149, 2150, 2151, 2152, 107, 2156, 2157, 110, 112, 115, 121, 126, 130, 136, 137, 2185, 140, 144, 145, 2195, 4247, 2202, 2203, 2206, 10399, 160, 12448, 164, 2217, 2218, 171, 172, 175, 179, 2231, 12471, 185, 190, 192, 2250, 2251, 204, 206, 214, 216, 221, 2270, 2271, 224, 227, 2281, 2285, 239, 2296, 2300, 254, 2302, 2303, 2305, 12542, 12543, 2308, 263, 2312, 265, 4362, 268, 269, 16652, 2319, 2321, 2323, 2324, 282, 12572, 289, 2338, 12578, 297, 2346, 300, 6445, 2350, 8494, 307, 12596, 311, 2360, 315, 2364, 4413, 318, 12609, 14657, 2374, 327, 2381, 12624, 346, 2404, 2406, 2407, 362, 2415, 4464, 16757, 2425, 2427, 381, 10622, 387, 399, 400, 2447, 2448, 404, 2453, 14741, 14746,

In [6]:
len(indices)

784

In [11]:
# Acceder a las filas usando iloc y la lista de índices
df[['name','address']].iloc[indices].reset_index()

,index,name,address
0,0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457"
1,6,Centro Botanero La Carreta,"Centro Botanero La Carreta, 1657 Main St, Gree..."
2,11,Mr Scott’s Tavern & Grill,"Mr Scott’s Tavern & Grill, W7318 WI-86, Tomaha..."
3,16,Valeo's Pizza,"Valeo's Pizza, 9755 76th St, Pleasant Prairie,..."
4,18,Ribs and Ribeyes,"Ribs and Ribeyes, W7457 County Hwy E, Westfiel..."
...,...,...,...
779,18435,ICHI Japanese Ramen House,7537 Maple St
780,18494,Provisions Gourmet Market,11615 Olive Blvd
781,18502,Angell's Bar & Grill,999 W Main St
782,18503,Tokiwa Ramen,11978 104 Avenue
